In [0]:
# Databricks notebook source
# ==========================================================
# CURATED - DATAENTRY EQUIPOS
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================

import env_setup
from pyspark.sql import SparkSession
from utils_liga1 import setup_adls, get_dbutils, get_abfss_path, read_parquet_adls, write_parquet_adls, is_dataframe_empty
from curated_dataentry import procesar_curated_dataentry

# ----------------------------------------------------------
# PARÁMETROS DESDE ADF
# ----------------------------------------------------------
dbutils.widgets.text("nombre_archivo", "")
dbutils.widgets.text("filesystem", "")
dbutils.widgets.text("capa_raw", "")
dbutils.widgets.text("rutaBase", "")
dbutils.widgets.text("ModoEjecucion", "")

nombre_archivo = dbutils.widgets.get("nombre_archivo")
filesystem = dbutils.widgets.get("filesystem")
capa_raw = dbutils.widgets.get("capa_raw").strip("/")
rutaBase = dbutils.widgets.get("rutaBase").strip("/")
ModoEjecucion = dbutils.widgets.get("ModoEjecucion")

# ----------------------------------------------------------
# CONFIGURACIÓN DE ADLS Y SPARK
# ----------------------------------------------------------
spark = SparkSession.builder.getOrCreate()
setup_adls()
dbutils = get_dbutils()

print("===============================================")
print("PROCESO CURATED DATAENTRY EQUIPOS")
print("===============================================")
print(f"Entidad        : {nombre_archivo}")
print(f"Modo ejecución : {ModoEjecucion}")
print("===============================================")

In [0]:
# ----------------------------------------------------------
# RUTAS DE LECTURA / ESCRITURA
# ----------------------------------------------------------
ruta_origen = f"{capa_raw}/{rutaBase}/{nombre_archivo}/stg/{nombre_archivo}/data"
ruta_destino = f"{capa_raw}/{rutaBase}/{nombre_archivo}/data"

ruta_abfss_origen = get_abfss_path(ruta_origen)
ruta_abfss_destino = get_abfss_path(ruta_destino)

# ----------------------------------------------------------
# PROCESAMIENTO
# ----------------------------------------------------------
try:
    print("===== LECTURA DESDE STG/DATA =====")
    df = read_parquet_adls(spark, ruta_abfss_origen)

    if is_dataframe_empty(df):
        raise Exception(f"No se encontró data en la ruta origen: {ruta_abfss_origen}")

    print("===== LIMPIEZA Y NORMALIZACIÓN =====")
    df_curated = procesar_curated_dataentry(df)

    print("===== ESCRITURA A DESTINO FINAL =====")
    # Sobrescribe completamente la data (FULL)
    write_parquet_adls(df_curated, ruta_abfss_destino)
    print(f"[OK] Archivo limpio escrito en {ruta_abfss_destino}")

    print("===== LIMPIEZA STG/DATA =====")
    dbutils.fs.rm(ruta_abfss_origen, recurse=True)
    print(f"[OK] Carpeta temporal eliminada: {ruta_abfss_origen}")

except Exception as e:
    print("[ERROR] Falló el proceso Curated Dataentry Equipos:", str(e))
    import traceback
    print(traceback.format_exc())
    raise